In [1]:
import numpy as np
import cv2
import time
import glob

from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

!pip install dv
import dv

from joblib import dump, load

In [2]:
def loadLabel(path):
    labels = np.genfromtxt(path, delimiter=',', skip_header=1, dtype=int)

    start_time = labels[0][1]
    for i in labels:
        i[1] = i[1]-start_time
        i[2] = i[2]-start_time
        
    return labels

In [3]:
# Return the start time of the given class index.
def start(index, labels):
    return labels[index][1]
 
# Return the end time of the given class index.
def end(index, labels):
    return labels[index][2]

def classID(index, labels):
    return labels[index][0]-1

# Return the class of the current frame. If no gesture then return -1.
def setGesture(currentTime, labels):
    for i in range(len(labels)-1):
        if (currentTime >= start(i, labels) and currentTime <= end(i, labels)):
            return classID(i, labels)
    return -1

In [4]:
def process(X, Y, path, labels):
    offset = 0
    
    with dv.AedatFile(path) as f: 
        for frame in f['frames']:    
            # Set the offset on the first frame.
            if (offset == 0):
                offset = frame.timestamp

            gesture = setGesture((frame.timestamp - offset), labels)
            image = frame.image

            # font
            font = cv2.FONT_HERSHEY_SIMPLEX

            # org
            org = (5, 30)

            # fontScale
            fontScale = 1

            # Blue color in BGR
            color = (255, 0, 0)

            # Line thickness of 2 px
            thickness = 2
            image = cv2.putText(image, str(gesture), org, font, 
                       fontScale, color, thickness, cv2.LINE_AA)
            
            # cv2.imshow('out',image)
            # cv2.waitKey(1)

            if (gesture == -1):
                continue
            else:
                Y.append(gesture)

                # TODO: Scale the data.
                matrix = frame.image.flatten()
                X.append(matrix)

In [5]:
def verify(csv, file):
    
    csv = csv.split("_labels")
    file = file.split(".")
    
    if (csv[0] != file[0]):
        print("mismatch: (%s, %s)" % (csv[0], file[0]))

In [6]:
X_train = []
Y_train = []

X_test = []
Y_test = []

split = 45

csv = sorted(glob.glob("data/*.csv"))
aedat4 = sorted(glob.glob("data/*.aedat4"))

for i in range(split):
    verify(csv[i], aedat4[i])
    print(csv[i], aedat4[i])
    
    labels = loadLabel(csv[i])
    process(X_train, Y_train, aedat4[i], labels)

print()
for i in range(split, split+5):
    verify(csv[i], aedat4[i])
    print(csv[i], aedat4[i])
    
    labels = loadLabel(csv[i])
    process(X_test, Y_test, aedat4[i], labels)

data\user01_fluorescent_labels.csv data\user01_fluorescent.aedat4
data\user01_fluorescent_led_labels.csv data\user01_fluorescent_led.aedat4
data\user01_lab_labels.csv data\user01_lab.aedat4
data\user01_led_labels.csv data\user01_led.aedat4
data\user01_natural_labels.csv data\user01_natural.aedat4
data\user02_fluorescent_labels.csv data\user02_fluorescent.aedat4
data\user02_fluorescent_led_labels.csv data\user02_fluorescent_led.aedat4
data\user02_lab_labels.csv data\user02_lab.aedat4
data\user02_led_labels.csv data\user02_led.aedat4
data\user02_natural_labels.csv data\user02_natural.aedat4
data\user03_fluorescent_labels.csv data\user03_fluorescent.aedat4
data\user03_fluorescent_led_labels.csv data\user03_fluorescent_led.aedat4
data\user03_led_labels.csv data\user03_led.aedat4
data\user03_natural_labels.csv data\user03_natural.aedat4
data\user04_fluorescent_labels.csv data\user04_fluorescent.aedat4
data\user04_fluorescent_led_labels.csv data\user04_fluorescent_led.aedat4
data\user04_led_

In [7]:
print(len(X_train))
print(len(Y_train))

print(len(X_test))
print(len(Y_test))

32751
32751
3661
3661


In [8]:
pipeline = make_pipeline(StandardScaler(), svm.SVC(kernel="rbf"))

pipeline.fit(X_train, Y_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('svc', SVC())])

In [9]:
pipeline.score(X_test, Y_test)

0.6200491668942911

In [10]:
# Save the model
dump(pipeline, 'pipeline.joblib') 

['pipeline.joblib']